In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('data.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df=df.dropna()

In [5]:
X=df.drop('label',axis=1)

In [6]:
y=df['label']

In [7]:
import tensorflow as tf

In [8]:
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from tensorflow.keras .preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential

In [9]:
V=5000

In [10]:
messages=X.copy()

In [11]:
messages.reset_index(inplace=True)

In [12]:
import nltk
import re

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
from nltk.corpus import stopwords

In [15]:
from nltk.stem.porter import PorterStemmer

In [16]:
def stem(df,title):
  corpus=[]
  ps=PorterStemmer()
  for i in range(len(df)):
    message=df[title][i]
    message=re.sub('[^a-zA-Z]',' ',message)
    message=message.lower()
    message=message.split(' ')
    message=[ps.stem(word) for word in message if word not in stopwords.words('english')]
    message=' '.join(message)
    corpus.append(message)
  return corpus

In [17]:
corpus=stem(messages,'title')

In [18]:
one_hot_repr=[one_hot(i,V) for i in corpus]

In [19]:
sent_len=20
padded_seq=pad_sequences(one_hot_repr,sent_len,padding='pre')

In [20]:
embedding_features=40
m1=Sequential()
m1.add(Embedding(V,embedding_features,input_length=sent_len))
m1.add(LSTM(100))
m1.add(Dense(1,activation='sigmoid'))
m1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(m1.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
import numpy as np

In [22]:
X_final=np.array(padded_seq)
y_final=np.array(y)

In [23]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33)

In [24]:
m1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 8s 33ms/step - loss: 0.3297 - accuracy: 0.8459 - val_loss: 0.1861 - val_accuracy: 0.9205
Epoch 2/10
192/192 [==============================] - 6s 31ms/step - loss: 0.1456 - accuracy: 0.9433 - val_loss: 0.1822 - val_accuracy: 0.9229
Epoch 3/10
192/192 [==============================] - 6s 31ms/step - loss: 0.1015 - accuracy: 0.9618 - val_loss: 0.2133 - val_accuracy: 0.9186
Epoch 4/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0691 - accuracy: 0.9756 - val_loss: 0.2309 - val_accuracy: 0.9226
Epoch 5/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0449 - accuracy: 0.9860 - val_loss: 0.2711 - val_accuracy: 0.9223
Epoch 6/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0290 - accuracy: 0.9909 - val_loss: 0.3427 - val_accuracy: 0.9254
Epoch 7/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0161 - accuracy: 0.9962 - val_loss: 0.4181 - val_accuracy: 0.9180

In [25]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [33]:
def results(y_test,y_pred):
  print("Confusion matrix: ",confusion_matrix(y_test,y_pred))
  print("\nAccuracy: ",accuracy_score(y_test,y_pred))

In [27]:
y_pred=m1.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [34]:
results(y_test,y_pred)

Confusion matrix:  [[3112  327]
 [ 155 2441]]

Accuracy:  0.9201325600662801


In [29]:
embedding_features=40
m2=Sequential()
m2.add(Embedding(V,embedding_features,input_length=sent_len))
m2.add(Dropout(0.3))
m2.add(LSTM(100))
m2.add(Dropout(0.3))
m2.add(Dense(1,activation='sigmoid'))
m2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(m1.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
m2.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 8s 34ms/step - loss: 0.3468 - accuracy: 0.8371 - val_loss: 0.1916 - val_accuracy: 0.9195
Epoch 2/10
192/192 [==============================] - 6s 32ms/step - loss: 0.1603 - accuracy: 0.9367 - val_loss: 0.1805 - val_accuracy: 0.9229
Epoch 3/10
192/192 [==============================] - 6s 32ms/step - loss: 0.1213 - accuracy: 0.9537 - val_loss: 0.1916 - val_accuracy: 0.9246
Epoch 4/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0963 - accuracy: 0.9645 - val_loss: 0.2129 - val_accuracy: 0.9181
Epoch 5/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0757 - accuracy: 0.9745 - val_loss: 0.2377 - val_accuracy: 0.9208
Epoch 6/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0572 - accuracy: 0.9813 - val_loss: 0.2486 - val_accuracy: 0.9181
Epoch 7/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0434 - accuracy: 0.9856 - val_loss: 0.3459 - val_accuracy: 0.9153

In [31]:
y_pred=m2.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [35]:
results(y_test,y_pred)

Confusion matrix:  [[3112  327]
 [ 155 2441]]

Accuracy:  0.9201325600662801
